# Library import

### download di YOLOv5 dalla repository github

In [ ]:
pip install -U ultralytics
pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

### import librerie

In [ ]:
import os

from PIL import Image
from torch.utils.data import Dataset

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from tqdm import tqdm


## Path

# Dataloader

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, txt_file, img_dir, utils, aug=False):
        """
        Dataset personalizzato per SSD che carica immagini e region proposals.
        
        Args:
            txt_file (str): File di testo con i percorsi delle immagini.
            img_dir (str): Directory contenente le immagini.
            utils: Funzioni di utilità caricate con torch.hub (nvidia_ssd_processing_utils).
            transforms (callable, optional): Trasformazioni da applicare alle immagini.
        """
        # Caricamento dei percorsi delle immagini
        with open(txt_file, 'r') as f:
            self.image_paths = [line.strip() for line in f.readlines()]
        
        self.img_dir = img_dir  # Directory delle immagini
        self.utils = utils  # Utilità di preprocessing (prepare_input, prepare_tensor)
        self.aug = aug  # Trasformazioni aggiuntive opzionali
        self.transform = transforms.Compose([
            transforms.Resize((320, 320)),
            transforms.ToTensor(),
        ]) # trasformazione di base da applicare a tutte le immagini
        
    def __len__(self):
        """
        Restituisce il numero di immagini nel dataset.
        """
        return len(self.image_paths)

    def __getitem__(self, index):
        # Ottieni il percorso dell'immagine
        img_name = os.path.basename(self.image_paths[index])
        img_path = os.path.join(self.img_dir, img_name)
        
        if not os.path.exists(img_path):
            raise FileNotFoundError(f"Immagine non trovata: {img_path}")
        
        # Carica l'immagine
        image = Image.open(img_path).convert('RGB')
        
        # Preprocessa l'immagine usando utils.prepare_input
        processed_image = self.utils.prepare_input([img_path])  # Restituisce lista di tensori
        
        # Converte in tensore
        image_tensor = self.utils.prepare_tensor(processed_image)
        
        # Applica trasformazioni opzionali
        if self.aug:
            image_tensor = self.transform(image_tensor)
        
        # Restituisci il tensore dell'immagine
        return {
            "image": image_tensor,  # Tensore preprocessato per SSD
            "path": img_path        # Percorso originale per riferimento
        }


In [ ]:
def collate_fn(batch):

    # Estrai le immagini e i percorsi dal batch
    images = [item["image"] for item in batch]
    paths = [item["path"] for item in batch]
    
    # Combina i tensori delle immagini in un unico tensore
    # (assumendo che tutte le immagini abbiano la stessa dimensione dopo il preprocessing)
    images_tensor = torch.cat(images, dim=0)
    
    return {
        "images": images_tensor,  # Batch di immagini come tensore
        "paths": paths            # Percorsi originali delle immagini
    }


In [ ]:
# Creazione dei dataset
train_dataset = CustomDataset(train_txt_pth, images_fldr_pth, utils, aug=True) 
valid_dataset = CustomDataset(val_txt_pth, images_fldr_pth, utils, aug=False)  
test_dataset = CustomDataset(test_txt_pth, images_fldr_pth, utils, aug=False)  

# Creazione dei DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

# Network

In [ ]:
# PER IL MODELLO YOLO
class YoloModel(nn.Module):
    def __init__(self, num_classes):
        super(YoloModel, self).__init__()

        ## Model -> per info https://github.com/NVIDIA/DeepLearningExamples/tree/master/PyTorch/Detection/SSD
        self.yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s', classes = 12,  autoshape = False, pretrained=True)

    def forward(self, images):

        # Calcola le previsioni con il modello SSD
        predictions = self.yolo_model(images)  # Output grezzo del modello SSD
        
        return predictions

In [ ]:
class SSDModel(nn.Module):
    def __init__(self, num_classes):
        super(SSDModel, self).__init__()

        ## Model -> per info https://github.com/NVIDIA/DeepLearningExamples/tree/master/PyTorch/Detection/SSD
        self.ssd_model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd') # modello pre-addestrato su dataset COCO
        self.utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd_processing_utils')

    def forward(self, images):

        # Calcola le previsioni con il modello SSD
        predictions = self.ssd_model(images)  # Output grezzo del modello SSD
        
        return predictions

In [ ]:
class Trainer:
    def __init__(self, model, train_loader, val_loader, criterion, optimizer=None, device='cuda'):
        """
        Inizializza la classe Trainer.
        
        Args:
            model: Il modello SSD.
            train_loader: DataLoader per il training set.
            val_loader: DataLoader per il validation set.
            criterion: Funzione di perdita.
            optimizer: Ottimizzatore (opzionale, di default Adam).
            device: Dispositivo per il calcolo ('cuda' o 'cpu').
        """
        self.model = model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.criterion = criterion
        self.optimizer = optimizer if optimizer else Adam(model.parameters(), lr=1e-4)
        self.device = device
    
    def train_one_epoch(self, epoch):
        """
        Esegue un'epoca di addestramento.
        """
        self.model.train()
        running_loss = 0.0
        pbar = tqdm(self.train_loader, desc=f"Training Epoch {epoch}")
        
        for images, targets in pbar:
            images, targets = images.to(self.device), targets.to(self.device)
            
            # Forward pass
            predictions = self.model(images)
            
            # Calcolo della perdita
            loss = self.criterion(predictions, targets)
            
            # Backward pass e ottimizzazione
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
            running_loss += loss.item()
            pbar.set_postfix({"loss": running_loss / len(self.train_loader)})
        
        return running_loss / len(self.train_loader)
    
    def validate_one_epoch(self, epoch):
        """
        Esegue un'epoca di validazione.
        """
        self.model.eval()
        running_loss = 0.0
        pbar = tqdm(self.val_loader, desc=f"Validation Epoch {epoch}")
        
        with torch.no_grad():
            for images, targets in pbar:
                images, targets = images.to(self.device), targets.to(self.device)
                
                # Forward pass
                predictions = self.model(images)
                
                # Calcolo della perdita
                loss = self.criterion(predictions, targets)
                running_loss += loss.item()
                
                pbar.set_postfix({"val_loss": running_loss / len(self.val_loader)})
        
        return running_loss / len(self.val_loader)
    
    def fit(self, epochs):
        """
        Esegue l'addestramento e la validazione per un dato numero di epoche.
        """
        train_losses = []
        val_losses = []
        
        for epoch in range(1, epochs + 1):
            train_loss = self.train_one_epoch(epoch)
            val_loss = self.validate_one_epoch(epoch)
            
            train_losses.append(train_loss)
            val_losses.append(val_loss)
            
            print(f"Epoch {epoch}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")
        
        return train_losses, val_losses

In [ ]:
ssd_model.to('cuda')

# Training

In [ ]:
# Inizializza il tuo modello
num_classes = 12  
ssd = SSDModel(num_classes)

# Addestra il modello, con validazione ad ogni epoca
ssd.train(train_loader, val_loader, num_epochs=1)

# Testing